# Helper for GRCh37_hg19 Universal BED creation

## Step 1

Juntar o MANE_Select e o MANE_Plus_clinical num só ficheiro.

In [ ]:
import pandas as pd
import os

# Definindo os caminhos dos arquivos
mane_plus_clinical_path = '/Users/ptpedfilven/Desktop/metrics_app/data/regions/universal_bed/GRCh38_hg38/MANE/original/MANE_Plus_clinical.bed'
mane_select_path = '/Users/ptpedfilven/Desktop/metrics_app/data/regions/universal_bed/GRCh38_hg38/MANE/original/MANE_Select.bed'
output_path = '/Users/ptpedfilven/Desktop/metrics_app/data/regions/universal_bed/GRCh38_hg38/MANE/original/MANE_Select_Plus_clinical.bed'

# Verificar se os arquivos existem
if not os.path.exists(mane_plus_clinical_path):
    print(f"Arquivo não encontrado: {mane_plus_clinical_path}")
    exit()

if not os.path.exists(mane_select_path):
    print(f"Arquivo não encontrado: {mane_select_path}")
    exit()

# Carregar os arquivos BED, especificando que as primeiras linhas são os cabeçalhos
mane_plus_clinical = pd.read_csv(mane_plus_clinical_path, sep='\t', header=0)
mane_select = pd.read_csv(mane_select_path, sep='\t', header=0)

# Realizar a junção sem sufixos
merged = pd.merge(
    mane_plus_clinical,
    mane_select,
    on=['Chromosome/scaffold name', 'Exon region start (bp)', 'Exon region end (bp)',
        'Gene stable ID version', 'Gene name', 'Exon stable ID', 'Strand', 'Transcript stable ID version'],
    how='outer'  # 'outer' para incluir todas as entradas de ambos os arquivos
)

# Ordenar pelo 'Exon region start (bp)'
merged_sorted = merged.sort_values(by='Exon region start (bp)')

# Salvar o arquivo resultante em um novo arquivo BED
merged_sorted.to_csv(output_path, sep='\t', header=True, index=False)  # Mantém o cabeçalho

print("A junção foi concluída e salva em 'MANE_Select_Plus_clinical.bed'.")


## Step 2 

Fazer download dos RefSeq para o MANE_Select e MANE_Plus_clinical e juntar tudo num json.

In [ ]:
import pandas as pd
import json

# Caminhos para os arquivos .txt
file1 = '/Users/ptpedfilven/Downloads/hg38_transcriptID_RefSeq_MANE_Plus_clinical.txt'  # Caminho do primeiro arquivo
file2 = '/Users/ptpedfilven/Downloads/hg38_transcriptID_RefSeq_MANE_Select.txt'  # Caminho do segundo arquivo

# Ler os dois arquivos .txt, considerando a primeira linha como cabeçalho
df1 = pd.read_csv(file1, sep='\t', header=0)  # header=0 é opcional, já que é o padrão
df2 = pd.read_csv(file2, sep='\t', header=0)

# Verificar e remover espaços em branco dos nomes das colunas
df1.columns = df1.columns.str.strip()
df2.columns = df2.columns.str.strip()

# Renomear as colunas relevantes
df1.rename(columns={'RefSeq match transcript (MANE Plus Clinical)': 'RefSeq'}, inplace=True)
df2.rename(columns={'RefSeq match transcript (MANE Select)': 'RefSeq'}, inplace=True)

# Converter os DataFrames em listas de dicionários
data1 = df1.to_dict(orient='records')
data2 = df2.to_dict(orient='records')

# Realizar o append dos dados do segundo arquivo ao primeiro
combined_data = data1 + data2

# Salvar o resultado em um arquivo JSON usando json.dump
with open('/Users/ptpedfilven/Downloads/hg38_transcriptID_RefSeq_MANE_Select_Plus_clinical.json', 'w') as json_file:
    json.dump(combined_data, json_file, indent=4)

print("Os dados foram appendidos e salvos como hg38_transcriptID_RefSeq_MANE_Select_Plus_clinical.json.")


Fazer o mesmo para o Ensembl hg19

In [ ]:
import pandas as pd
import json

# Caminho do arquivo
file = '/Users/ptpedfilven/Downloads/hg37_transcriptID_RefSeq.txt'

# Ler o arquivo .txt, considerando a primeira linha como cabeçalho
df = pd.read_csv(file, sep='\t', header=0)

# Verificar e remover espaços em branco dos nomes das colunas
df.columns = df.columns.str.strip()

# Renomear as colunas relevantes
df.rename(columns={"RefSeq mRNA ID": 'RefSeq'}, inplace=True)

# Remover as linhas onde 'RefSeq' é NaN
df = df.dropna(subset=['RefSeq'])

# Converter o DataFrame em uma lista de dicionários
data = df.to_dict(orient='records')

# Salvar o resultado em um arquivo JSON usando json.dump
with open('/Users/ptpedfilven/Downloads/hg37_transcriptID_RefSeq.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

print("Os dados foram processados e salvos como hg37_transcriptID_RefSeq.json.")


In [ ]:
python bed_to_json.py ../../data/regions/universal_bed/GRCh38_hg38/MANE/original/MANE_Select_Plus_clinical.bed ../../data/regions/universal_bed/GRCh38_hg38/MANE/original/MANE_Select_Plus_clinical_v2.json --refseq_file ../../data/regions/universal_bed/GRCh38_hg38/aux_MANE_match/hg38_transcriptID_RefSeq_MANE_Select_Plus_clinical.json

In [ ]:
python bed_to_json.py ../../data/regions/universal_bed/GRCh37_hg19/original/Ensembl.bed ../../data/regions/universal_bed/GRCh37_hg19/original/Ensembl_v2.json --refseq_file ../../data/regions/universal_bed/GRCh37_hg19/aux_MANE_match/hg37_transcriptID_RefSeq.json

In [ ]:
python universal_bed.py ../../data/regions/universal_bed/GRCh37_hg19/original/Ensembl_v2.json ../../data/regions/universal_bed/GRCh37_hg19/modified/MANE_Select_Plus_clinical_hg37_v2.bed hg37 --mane_file ../../data/regions/universal_bed/GRCh38_hg38/MANE/original/MANE_Select_Plus_clinical_v2.json